# Introduction to google Colab and yFinance.

Good morning all, I am just putting together this quick notebook so that we can reference it if we need to. This will allow you to get comfortable with the yfinance API and it will also allow us to figure out how to connect to the shared network with our notebooks. The fact that the Machine Learning Pipelines is one of the prereq's I imagine we should use the markfile setup with multiple python scripts that can be run alone or run together with the if __name__ == "__main__": command. So let's get started with connecting.

In [ ]:
# Just going to put an easy dynamic variable to change with whoever is working in the notebook

user = 'Tom'

## Connecting Colab to Shared Drive
We will start by adding a few libraries and then setting the file path. The file path will likely be different for all of us so I will make it dynamic and you can put in the pathway you have and we can just switch out the names.

In [ ]:
# importing only the connection libraries
import os
from google.colab import drive
drive.mount('/content/drive')

#Specify paths specific to the user
if user.lower() == 'tom':
    path = '/content/drive/MyDrive/Colab Notebooks/696 - Milestone II/696 - Milestone II - Shared'
if user.lower() == 'peter':
    path = ''
if user.lower() == 'melody':
    path = ''

#Change to the directory for the shared network
os.chdir(path)
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/Colab Notebooks/696 - Milestone II/696 - Milestone II - Shared'

In [ ]:
#Let's make sure its connected to the right one by listing some files in the directory
os.listdir()

["Yueyao's idea", 'Intro to yfinance.ipynb']

## Introduction to yfinance
yfinance is relatively easy to use and one of the most popular free sources of stock data. I used it in milestone I to obtain price data for thousands of stocks (be careful of rate limiting) and I am excited to explore more of the fundamental side of the API with this project. I'll show you a very quick and easy example but feel free to check out the documentation [here](https://https://ranaroussi.github.io/yfinance/).


In [ ]:
# Import the libraries for working with yfinance
import numpy as np
import pandas as pd
import yfinance as yf
import altair as alt

In [ ]:
#Let's start with a really popular ticker for Microsoft
'''Just as a side note, a ticker is the stock symbol, which is typically a three to four letter code to identify the stock on the stock exchange. We will be dealing with them a lot just so you get the terminology'''

#Assign our ticker
msft = yf.Ticker('MSFT')

#Let's start by getting some price data. Let's do the past three weeks in 5' intervals (it will give us the date/time, high, low, open, close and volume for every 5 mins of trading over the past three weeks)
msft_price = yf.download(tickers = 'MSFT', period = '21d', interval = '5m')
msft_price.head()

/tmp/ipython-input-2288205671.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  msft_price = yf.download(tickers = 'MSFT', period = '21d', interval = '5m')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
Datetime,,,,,
2025-08-01 13:30:00+00:00,531.289978,535.799988,530.320007,535.315002,2232960
2025-08-01 13:35:00+00:00,530.695007,531.859985,530.000000,531.289978,577468
2025-08-01 13:40:00+00:00,530.655029,532.604980,530.200012,530.659973,713810
2025-08-01 13:45:00+00:00,529.780029,530.859985,528.989990,530.655029,559370
2025-08-01 13:50:00+00:00,528.909973,530.089905,528.470093,529.799988,477935


Great, so it was that simple to get historical price data for a stock. Now, let's get some fundamental information like the sector, market cap, etc. These are things that we will likely want to add to the dataframe for our machine learning pipeline. so it's pretty easy to pull the info we want. Let's just look at a few.

In [ ]:
#Get the corporate info for Microsoft
msft_info = msft.info
#Let's print off some easy information
print(f'Sector: {msft_info.get('sector')}')
print(f'Industry: {msft_info.get('industry')}')
print(f'Market Capitalization: {msft_info.get('marketCap')}')
print(f'Employees: {msft_info.get('fullTimeEmployees')}')

Sector: Technology
Industry: Software - Infrastructure
Market Capitalization: 3781996838912
Employees: 228000


Now, for the big part of our work. We are going to be working a lot with financail statements in order to model revenue, expenses, margin, and other health indicators for a company moving forward so that we can predict a price target for the share price in the future. So let's grab some annual statements and some quarterly statements.

In [ ]:
#Let's pull some financial information from Microsoft
annual_statements = msft.get_income_stmt(freq='yearly')
quarterly_statements = msft.get_income_stmt(freq='quarterly')

Now, let's take a look at both of these dataframes. We will need to figure out which metrics are good, which are likely unimportant (or highly correlated with other metrics). We can then also think about normalization (percentage of revenue) so that companies are comparable regardless of market cap and a way of bring in timeseries analysis (using year over year change instead of just the raw values alone). These are just my thoughts so far.

In [ ]:
annual_statements

,2025-06-30,2024-06-30,2023-06-30,2022-06-30
TaxEffectOfUnusualItems,-7.721784e+07,-1.000900e+08,-2.850000e+06,4.375400e+07
TaxRateForCalcs,1.762960e-01,1.823130e-01,1.900000e-01,1.310000e-01
NormalizedEBITDA,1.606030e+11,1.335580e+11,1.051550e+11,9.990500e+10
TotalUnusualItems,-4.380000e+08,-5.490000e+08,-1.500000e+07,3.340000e+08
TotalUnusualItemsExcludingGoodwill,-4.380000e+08,-5.490000e+08,-1.500000e+07,3.340000e+08
NetIncomeFromContinuingOperationNetMinorityInterest,1.018320e+11,8.813600e+10,7.236100e+10,7.273800e+10
ReconciledDepreciation,3.415300e+10,2.228700e+10,1.386100e+10,1.446000e+10
ReconciledCostOfRevenue,8.783100e+10,7.411400e+10,6.586300e+10,6.265000e+10
EBITDA,1.601650e+11,1.330090e+11,1.051400e+11,1.002390e+11
EBIT,1.260120e+11,1.107220e+11,9.127900e+10,8.577900e+10


Let's look at the revenue for the past four years and see if it is growing and where we would forecast it to go just by eye.

In [ ]:
plotting_df = annual_statements.T
plotting_df.reset_index(inplace=True, names = 'date')
#plotting_df.head()
rev = alt.Chart(plotting_df).mark_bar(width=20, color = 'black').encode(x='date:T', y='TotalRevenue:Q')
rev.show()

alt.Chart(...)

In [ ]:
quarterly_statements

,2025-06-30,2025-03-31,2024-12-31,2024-09-30,2024-06-30
TaxEffectOfUnusualItems,4.951251e+05,6.966000e+07,-2.032200e+08,5.719000e+07,-1.626385e+07
TaxRateForCalcs,1.650420e-01,1.800000e-01,1.800000e-01,1.900000e-01,1.913390e-01
NormalizedEBITDA,4.443100e+10,4.032400e+10,3.791500e+10,3.793300e+10,3.441600e+10
TotalUnusualItems,3.000000e+06,3.870000e+08,-1.129000e+09,3.010000e+08,-8.500000e+07
TotalUnusualItemsExcludingGoodwill,3.000000e+06,3.870000e+08,-1.129000e+09,3.010000e+08,-8.500000e+07
NetIncomeFromContinuingOperationNetMinorityInterest,2.723300e+10,2.582400e+10,2.410800e+10,2.466700e+10,2.203600e+10
ReconciledDepreciation,1.120300e+10,8.740000e+09,6.827000e+09,7.383000e+09,6.380000e+09
ReconciledCostOfRevenue,2.401400e+10,2.191900e+10,2.179900e+10,2.009900e+10,1.968400e+10
EBITDA,4.443400e+10,4.071100e+10,3.678600e+10,3.823400e+10,3.433100e+10
EBIT,3.323100e+10,3.197100e+10,2.995900e+10,3.085100e+10,2.795100e+10


Okay, I think that is a good introduction and hopefully it gives you something to play around with and look at. Let me know if you have any questions!